# Chaining search


## Library functions

UI:

In [5]:

import ipywidgets as widgets
from IPython.display import display
DEFAULT_QUERY = r'[lemma="boek"]'
DEFAULT_CORPUS = "chn"

def create_corpus_ui():
    # Create UI elements
    corpusQueryField = widgets.Text(description="<b>CQL query:</b>", value=DEFAULT_QUERY)
    corpusField = widgets.Dropdown(
        options=['chn', 'opensonar', 'zeebrieven', 'gysseling', 'nederlab'],
        value=DEFAULT_CORPUS,
        description='<b>Corpus:</b>',
    )
    '''corpusSearchButton = widgets.Button(
        description='Search',
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Search',
    )
    # Handle events
    corpusSearchButton.on_click(corpus_search)'''

    # Stack UI elements in vertical box and display
    corpusUiBox = widgets.VBox([corpusQueryField,corpusField])
    display(corpusUiBox)
    
    # Return fields, so their contents are accessible from the global namespace of the Notebook
    return corpusQueryField, corpusField

def create_lexicon_ui():
    DEFAULT_SEARCHWORD = 'boek'
    DEFAULT_LEXICON = "diamant"

    # Create UI elements
    searchWordField = widgets.Text(description="<b>Word:</b>", value=DEFAULT_SEARCHWORD)
    lexiconField = widgets.Dropdown(
        options=['diamant'],
        value=DEFAULT_LEXICON,
        description='<b>Lexicon:</b>',
    )
    '''lexSearchButton = widgets.Button(
        description='Search',
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Search',
    )
    # Handle events
    lexSearchButton.on_click(lexicon_search)'''
    # Stack UI elements in vertical box and display
    lexUiBox = widgets.VBox([searchWordField,lexiconField])
    display(lexUiBox)
    return searchWordField, lexiconField

Queries:

In [6]:

def get_query(word, lexicon):
    if (lexicon=="anw"):
        subpart = 'FILTER ( regex(?lemma, "'+word+'") || regex(?definition, "'+word+'") ) .\n'
        query = """PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>\n
                  PREFIX anw: <http://rdf.ivdnt.org/lexica/anw>\n
                  PREFIX anwsch: <http://rdf.ivdnt.org/schema/anw/>\n
                  PREFIX lemon: <http://lemon-model.net/lemon#>\n
                  \n
                  SELECT ?lemId ?lemma ?writtenForm ?definition ?definitionComplement\n
                  FROM <http://rdf.ivdnt.org/lexica/anw>\n
                  WHERE {\n
                      ?lemId rdfs:label ?lemma .\n
                      ?lemId ontolex:sense ?senseId .\n
                      ?senseId lemon:definition ?definitionId .\n
                      ?definitionId lemon:value ?definition .\n
                      OPTIONAL { ?definitionId anwsch:definitionComplement ?definitionComplement .}\n
                      OPTIONAL { ?lemId ontolex:canonicalForm ?lemCFId . \n
                          ?lemCFId ontolex:writtenRepresentation ?writtenForm . }\n
                      """+subpart+"""\n
                      }\n"""
    elif (lexicon=="diamant"):
        query = """
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        prefix prov: <http://www.w3.org/ns/prov#>
        prefix diamant: <http://rdf.ivdnt.org/schema/diamant#>
        prefix lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        prefix lemon: <http://lemon-model.net/lemon#>
        prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#>
        prefix ud: <http://universaldependencies.org/u/pos/>
        prefix skos: <http://www.w3.org/2004/02/skos/core#>
        prefix dcterms: <http://purl.org/dc/terms/>
        prefix dc: <http://purl.org/dc/terms/>

        select ?n_entry ?n_form ?n_ontolex_writtenRep ?n_syndef ?n_sensedef ?n_sensedef_definitionText ?n_syndef_definitionText ?n_sense ?inputMode ?wy_f_show ?wy_t_show
        where
        {
        graph ?g
        {
        {
            { ?n_form ontolex:writtenRep ?n_ontolex_writtenRep .
              values ?n_ontolex_writtenRep  { \"""" + word + """\" } } .
            { ?n_entry a ontolex:LexicalEntry} .
            { ?n_form a ontolex:Form} .
            { ?n_sense a ontolex:LexicalSense} .
            { ?n_syndef a diamant:SynonymDefinition} .
            { ?n_sensedef a lemon:SenseDefinition} .
            { ?n_syndef diamant:definitionText ?n_syndef_definitionText } .
            { ?n_sensedef diamant:definitionText ?n_sensedef_definitionText } .
            { ?n_entry ontolex:canonicalForm ?n_form } .
            { ?n_entry ontolex:sense ?n_sense } .
            { ?n_sense lemon:definition ?n_syndef } .
            { ?n_sense lemon:definition ?n_sensedef } .
              ?n_sense diamant:attestation ?n_attest_show .
              ?n_sense diamant:attestation ?n_attest_filter .
              ?n_attest_show diamant:text ?n_q_show .
              ?n_attest_filter diamant:text ?n_q_filter .
              ?n_attest_show a diamant:Attestation .
              ?n_attest_filter a diamant:Attestation .
              ?n_q_filter a diamant:Quotation .
              ?n_q_show a diamant:Quotation .
              ?n_q_filter diamant:witnessYearFrom ?wy_f_filter .
              ?n_q_filter diamant:witnessYearTo ?wy_t_filter .
              ?n_q_show diamant:witnessYearFrom ?wy_f_show .
              ?n_q_show diamant:witnessYearTo ?wy_t_show .
              FILTER (xsd:integer(?wy_f_show) >= 1200)
              FILTER (xsd:integer(?wy_t_show) >= 1200)
              FILTER (xsd:integer(?wy_f_show) <= 2018)
              FILTER (xsd:integer(?wy_t_show) <= 2018)
            { bind("lemma" as ?inputMode) } .
            } UNION
          {
            { ?n_syndef diamant:definitionText ?n_syndef_definitionText .
            values ?n_syndef_definitionText  { \"""" + word + """\" } } .
            { ?n_sense a ontolex:LexicalSense} .
            { ?n_syndef a diamant:SynonymDefinition} .
            { ?n_sensedef a lemon:SenseDefinition} .
            { ?n_form a ontolex:Form} .
            { ?n_form ontolex:writtenRep ?n_ontolex_writtenRep } .  { ?n_entry a ontolex:LexicalEntry} .
            { ?n_entry ontolex:sense ?n_sense } .
            { ?n_sense lemon:definition ?n_syndef } .
            { ?n_sense lemon:definition ?n_sensedef } .
            { ?n_sensedef diamant:definitionText ?n_sensedef_definitionText } .
            { ?n_entry ontolex:canonicalForm ?n_form } .
            ?n_sense diamant:attestation ?n_attest_show .
            ?n_sense diamant:attestation ?n_attest_filter .
            ?n_attest_filter diamant:text ?n_q_filter .
            ?n_attest_show diamant:text ?n_q_show .
            ?n_q_filter diamant:witnessYearFrom ?wy_f_filter .
            ?n_q_filter diamant:witnessYearTo ?wy_t_filter .
            ?n_q_show diamant:witnessYearFrom ?wy_f_show .
            ?n_q_show diamant:witnessYearTo ?wy_t_show .
            ?n_attest_show a diamant:Attestation .
            ?n_attest_filter a diamant:Attestation .
            ?n_q_filter a diamant:Quotation .
            ?n_q_show a diamant:Quotation .
            FILTER (xsd:integer(?wy_f_show) >= 1200)
            FILTER (xsd:integer(?wy_t_show) >= 1200)
            FILTER (xsd:integer(?wy_f_show) <= 2018)
            FILTER (xsd:integer(?wy_t_show) <= 2018)
          { bind("defText" as ?inputMode) } .
            }
        }
        }"""

        return query

Search:

In [12]:

import requests
import pandas as pd
import xml.etree.ElementTree as ET
import json

def parse_xml(text):
    # TODO: should we secure against untrusted XML?
    root = ET.fromstring(text)
    records = []
    computed_nwih = False
    for entry in root.iter("{http://clarin.eu/fcs/resource}ResourceFragment"):
        for dataView in entry.findall("{http://clarin.eu/fcs/resource}DataView"):
            # We only take into account hits, ignore metadata and segmenting dataViews
            if (dataView.get("type")=="application/x-clarin-fcs-hits+xml"):
                result = dataView.find("{http://clarin.eu/fcs/dataview/hits}Result")
                left_context = result.text if result.text is not None else ''
                hits = list(result)
                last_hit = hits[-1]
                right_context = last_hit.tail if last_hit.tail is not None else ''
                hit_words = [hit.text for hit in hits]
                
                if not computed_nwih:
                    n_words_in_hit = len(hits)
                    computed_nwih=True
                kwic = [left_context] + hit_words + [right_context]
                records.append(kwic)
    columns = ["left context"] + ["word " + str(n) for n in range(n_words_in_hit)] + ["right context"]
    return pd.DataFrame(records, columns = columns)

def search_corpus(query, corpus):
    # Do request to federated content search corpora, so we get same output format for every corpus
    url = "http://portal.clarin.inl.nl/fcscorpora/clariah-fcs-endpoints/sru?operation=searchRetrieve&queryType=fcs&x-fcs-context=" + corpus + "&maximumRecords=20&query=" + query
    response = requests.get(url)
    response_text = response.text
    df = parse_xml(response_text)
    return df
   
def search_lexicon(query,corpus):
    endpoint = "http://svprre02:8080/fuseki/tdb/sparql"
    url = endpoint #+ "?query=" + query
    response = requests.post(url,data={"query":query})
    response_json = json.loads(response.text)
    records_json = response_json["results"]["bindings"]
    records_string = json.dumps(records_json)
    df = pd.read_json(records_string, orient="records")
    df = df.applymap(lambda x: x["value"])
    return df

## Corpus search

* Run the cell below to show the UI, and fill in your search query

In [9]:
#from chaininglib import ui

# Create corpus UI, creates references to field contents
corpusQueryField, corpusField = create_corpus_ui()

 * Run the cell below to search

In [6]:
#from chaininglib import search

query= corpusQueryField.value
corpus = corpusField.value
def test(a="a", b="b"):
    return function
df_corpus = search_corpus(query,corpus, limit=1000, option=b)
search().corpus("chn").option(a).execute()
display(df_corpus)



,left context,word 0,right context
0,of Imam Zij staat te,boek,als het eerste zwarte topmodel
1,microhandel in het land gestimuleerd,Boeken,met thema's die betrekking hebben
2,thema's Surinaamse opinie tijdschriften en,boeken,over winticultuur behoren ook tot
3,mee voorop lopen In zijn,boek,Verloren in wanorde geeft Karel
4,onafhankelijk forensische onderzoek naar de,boeken,te starten Dit moet een
5,al eerder gepubliceerd een nieuw,boek,van Orlando Emanuels is haast
6,104 bladzijden geen erg dik,boek,geworden maar het ziet er
7,vaak hoe je een goed,boek,kan onderscheiden Ik legde dan
8,gekscherend aan toe Een goed,boek,loopt slecht af En slecht
9,heb vaker met hem gesproken,boeken,aangehaald tijdschriften etc. etc. maar


## Lexicon search

* Run the cell below to show the UI, and fill in your search query in the UI

In [2]:
#from chaininglib import ui
searchWordField, lexiconField = ui.create_lexicon_ui()

 * Run the cell below to search

In [10]:
#from chaininglib import queries, search

search_word = searchWordField.value
lexicon = lexiconField.value
# USER: can replace this by own custom query
query = get_query(word=search_word, lexicon=lexicon)

df_lexicon = search_lexicon(query, lexicon)
df_lexicon_relevant = df_lexicon[["inputMode", "n_ontolex_writtenRep", "n_syndef_definitionText", "n_sensedef_definitionText", "wy_f_show", "wy_t_show"]]
display(df_lexicon_relevant)

,inputMode,n_ontolex_writtenRep,n_syndef_definitionText,n_sensedef_definitionText,wy_f_show,wy_t_show
0,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
1,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
2,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
3,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
4,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
5,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
6,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
7,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
8,lemma,boek,Boek,Boek.,1460,1480
9,lemma,boek,Boek,Boek.,1460,1480


## Case study: Frequency of *puur*+verb and *zuiver*+verb compared
* Below cell searches for *puur*+verb and for *zuiver*+verb in the CHN corpus
* Compare frequencies

In [11]:
#from chaininglib import search
from IPython.core.display import display, HTML

# Word 1: puur
query1= r'[word="puur"][pos="verb"]'
corpus1 = "chn"
df_corpus1 = search_corpus(query1,corpus1)
display(HTML('<b>puur</b>'))
display(df_corpus1)

# Word 2: zuiver
query2= r'[word="zuiver"][pos="verb"]'
corpus2 = "chn"
df_corpus2 = search_corpus(query2,corpus2)
display(HTML('<b>zuiver</b>'))
display(df_corpus2)

# Compute difference
set_df1 = set(df_corpus1["word 1"])
set_df2 = set(df_corpus2["word 1"])
# Elements of 1 that are not in 2
diff_left = set_df1.difference(set_df2)
display(HTML('Werkwoorden voor <b>puur</b> niet in <b>zuiver</b>: ' + ", ".join(diff_left)))
# Elements of 2 that are not in 1
diff_right = set_df2.difference(set_df1)
display(HTML('Werkwoorden voor <b>zuiver</b> niet in <b>puur</b>: ' + ", ".join(diff_right)))

,left context,word 0,word 1,right context
0,stad omdat de cultuur daar,puur,is,
1,succes dan wanneer de coalitie,puur,gestoeld,is op een parlementaire meerderheid
2,de andere jongeren die gewoon,puur,willen,werken Maar financieel zijn zij
3,gratis zijn De deelname is,puur,gebaseerd,op interesse In de middaguren
4,de natuur De natuur is,puur,vertelt,de kunstenaar Een tijger jaagt
5,van een licentieovereenkomst maar het,puur,gaat,om de oorspronkelijke eis van
6,gevecht voor ons voorouderlijk land,puur,bedoeld,is om het menselijk leven
7,het SZF maar waar het,puur,gaat,om winsten te maken zegt
8,redactrice Inge SchelstraeteHET zou poëzie,puur,worden,waarschuwde Piet Piryns ons Maar
9,bepaalde zender te adverteren is,puur,gebaseerd,op marketing én op het


,left context,word 0,word 1,right context
0,baby Ik wil het contact,zuiver,beperken,tot de baby en het
1,zo zul je als mens,zuiver,moeten,zijn om in aanmerking te
2,adviezen van mensen Ik heb,zuiver,vastgelegd,wat ik zelf spraakmakend vond
3,karaoke je ding Daarvoor is,zuiver,zingen,niet nodig Dat is juist
4,aan als het gevoel erachter,zuiver,is,Met deze woorden besloot Loes
5,Maar als je je gevoelens,zuiver,houdt,en goede wensen creëert dan
6,Cronie maar die was te,zuiver,genomen,Bij The Scorpions benutten Misiedjan
7,Woudman op Cairo Deze werd,zuiver,genomen,door Dwight Tempico 2-1 Een
8,de leerlingen herkenbaar en is,zuiver,beweert,de directeur Ook enkele leerkrachten
9,bakzeil De strafschop was te,zuiver,genomen,
